In [6]:
pip install twitter

     |████████████████████████████████| 61kB 4.0MB/s 


In [7]:
pip install python-twitter

     |████████████████████████████████| 71kB 4.0MB/s 


In [8]:
pip install twython

In [0]:
import tweepy ## For twitter API accessing
import pandas as pd
import numpy as np
import re, os, sys, datetime, twitter

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tweepy import OAuthHandler
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from IPython.display import clear_output

#Twitter API Authentication variables
# 1. get the verification of twitter developer
# 2. Create an APP in https://developer.twitter.com/en/apps/create
#    - APP name : tabiiApp
# 3. get the API key

accessToken = "1108257100809961472-5oysN7Jrr14cW3jf4M3LI0LbTtkFej"
accessTokenSecret = "kn0myr7w4whJxolkhLu7hr39HIUCsi52uxsefsPzMreiy"
consumerKey = "pn8yOn997NzXP7VUjCwPdFtwz"
consumerSecret = "26WA8DvSPxjKzlPG1yndKkM96PQl51tQZfzmeueKesH6OpglJ6"

In [0]:
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit= True)

In [0]:
search_words = "Frozen2" 
date_since = "2018-11-16"

#Collect tweets
tweets = tweepy.Cursor(api.search, 
                       q = '\"{}\" -filter:retweets'.format(search_words),
                       lang = "en",
                       since = date_since).items(9000)

results = []

for tweet in tweets:
  print(tweet.text)
  results.append(tweet)

Been obsessed with the #Frozen2 soundtrack since seeing the film literally listen to it on repeat every day! I hone… https://t.co/EG3k4uR7zZ
Loved the story for #Frozen2 but felt the same charm was not present as it was in the first one.  The tunes were no… https://t.co/RuKzEm86OB
oh gawd i'm such a softie for elsa. cutie babey. love you mami ;-; #frozen2 #elsa https://t.co/LBvsQxorD2
#Frozen2 death Disney on Ice; the cold hard facts of freezing to death rated XXX or NC-17... #parody https://t.co/fCjX8gDzCo
Disney's Frozen "Let It Go" Sequence Performed by Idina Menzel https://t.co/ooiFAiKLkr via @YouTube. So one of my l… https://t.co/frkDxa7nBq
#frozen2 left me like 😱😱
Hey ho
Do you wanna some more Anna and Elsa content in Twitter? Lol

#Frozen2 #punk #illustration https://t.co/Q5JIle78Xu
One of the best parts of #Frozen2 https://t.co/YOYxsyYKff
One of the thing that i really3 like frozen is the script!! Im so touched with how anna's talk to elsa. How she exp… https://t.co/JvPHqHg3pG


**Create a dataframe**

In [0]:
#Store tweets data in a dataframe

def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    data_set["Review"] = [tweet.text for tweet in results] #Text
    data_set["Created_at"] = [tweet.created_at for tweet in results]
    data_set["Retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["User_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["User_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["User_location"] = [tweet.author.location for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    
    return data_set

data_set = tweets_df(results)

Remove the duplicated text

In [0]:
text = data_set["Review"]

for i in range(0,len(text)):
    txt = ' '.join(word for word in text[i] .split() if not word.startswith('https:'))
    data_set.set_value(i, 'text', txt)
    
data_set.drop_duplicates('text', inplace=True)
data_set.reset_index(drop = True, inplace=True)
data_set.drop('Review', axis = 1, inplace = True)
data_set.rename(columns={'text': 'Review'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


Drop the unwanted data

In [0]:
data_set = data_set.drop(['id',
                      'Created_at',
                      'Retweet_count',
                      'User_screen_name',
                      'User_followers_count',
                      'User_location', 
                      'Hashtags'], axis = 1)

Save the data to csv

In [0]:
data_set.to_csv('data.csv')